# Ensembl genes table extraction EDA

This notebook is useful for development as well as exploratory data analysis on the extracted tables.
It is currently automically executed and saved as part of exports using `papermill`.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from ensembl_genes import ensembl_genes
from bioregistry import normalize_prefix

In [3]:
# parameters cell
species = "human"
release = "104"

In [4]:
# Parameters
species = "mouse"
release = "105"


In [5]:
ensg = ensembl_genes.Ensembl_Gene_Queries(release=release, species=species)
ensg.connection_url

'mysql+mysqlconnector://anonymous@ensembldb.ensembl.org:3306/mus_musculus_core_105_39'

In [6]:
database = ensg.database
database

'mus_musculus_core_105_39'

## Extract data

## gene attrib counts

In [7]:
ensg.run_query("gene_attrib_counts").head(15)

,attrib_type_id,code,name,description,attrib_type_count,attrib_type_examples
0,142,GeneGC,Gene GC,Percentage GC content for this gene,55414,"54.27, 50.52, 49.78, 50.40, 51.67, 53.41, 48.8..."
1,4,name,Name,Alternative/long name,49006,"RP24-365I5.2, RP23-455B5.4, RP23-455B5.3, Ppcs..."
2,395,xref_id,Xref ID,ID of associated database reference,48970,"OTTMUSG00000055405, OTTMUSG00000051993, OTTMUS..."
3,538,legacy_biotype,Legacy biotype,Obsolete biotype previously assigned to this E...,9949,"processed_transcript, lincRNA"
4,380,havana_cv,Havana CV term,Controlled vocabulary terms from Havana,5186,"ncRNA host, overlapping locus, retrogene, semi..."
5,54,remark,Remark,Annotation remark,1113,"confirm experimentally, 5730536A07Rik, miRNA G..."
6,382,NoTransRefError,No translations due to reference error,This gene is believed to include protein codin...,5,1
7,1,embl_acc,European Nucleotide Archive (was EMBL) accession,ENA,0,None
8,2,status,Status,None,0,None
9,3,synonym,Synonym,None,0,None


## genes

In [8]:
ensg.gene_df.head()

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
0,ENSMUSG00000000001,5,Gnai3,MGI,MGI:95773,protein_coding,ensembl_havana,2004-11-18 15:05:57,2020-08-10 14:30:16,GRCm39,...,108014596,108053462,-1,True,NaN,None,guanine nucleotide binding protein (G protein)...,MGI Symbol,MGI:95773,ENSMUSG00000000001
1,ENSMUSG00000000003,16,Pbsn,MGI,MGI:1860484,protein_coding,ensembl_havana,2006-06-28 13:29:12,2020-08-10 14:24:33,GRCm39,...,76881507,76897229,-1,True,NaN,None,probasin,MGI Symbol,MGI:1860484,ENSMUSG00000000003
2,ENSMUSG00000000028,16,Cdc45,MGI,MGI:1338073,protein_coding,ensembl_havana,2007-06-26 09:54:29,2020-08-10 14:17:14,GRCm39,...,18599197,18630737,-1,True,NaN,None,cell division cycle 45,MGI Symbol,MGI:1338073,ENSMUSG00000000028
3,ENSMUSG00000000031,17,H19,MGI,MGI:95891,lncRNA,ensembl_havana,2006-02-02 11:27:21,2020-08-10 14:39:18,GRCm39,...,142129266,142131880,-1,True,NaN,None,"H19, imprinted maternally expressed transcript",MGI Symbol,MGI:95891,ENSMUSG00000000031
4,ENSMUSG00000000037,18,Scml2,MGI,MGI:1340042,protein_coding,ensembl_havana,2006-08-18 16:16:32,2020-08-10 14:22:47,GRCm39,...,159865521,160041209,1,True,NaN,None,Scm polycomb group protein like 2,MGI Symbol,MGI:1340042,ENSMUSG00000000037


In [9]:
# clone-based genes no longer get a symbol and are filled with the stable ID
# https://www.ensembl.info/2021/03/15/retirement-of-clone-based-gene-names/
ensg.gene_df.query("gene_symbol == ensembl_gene_id").head(2)

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
13958,ENSMUSG00000047383,9,ENSMUSG00000047383,None,None,lncRNA,havana,2014-01-15 15:34:24,2020-08-10 14:18:26,GRCm39,...,59036282,59042698,-1,True,NaN,None,novel transcript,NaN,NaN,ENSMUSG00000047383
20360,ENSMUSG00000073879,5,ENSMUSG00000073879,None,None,unprocessed_pseudogene,havana,2009-11-16 12:20:32,2020-08-25 17:24:28,GRCm39,...,42236859,42239182,1,True,NaN,None,"family with sequence similarity 205, pseudogene",NaN,NaN,ENSMUSG00000073879


In [10]:
# which external database the gene symbol derives from versus the ensembl source
pd.crosstab(
    ensg.gene_df.ensembl_source,
    ensg.gene_df.gene_symbol_source_db.fillna("missing (clone-based)"),
    margins=True,
)

gene_symbol_source_db,EntrezGene,HGNC,MGI,RFAM,miRBase,missing (clone-based),All
ensembl_source,,,,,,,
RefSeq,0,0,37,0,0,0,37
ensembl,7,12,2975,856,13,1322,5185
ensembl_havana,4,1,22422,0,0,6,22433
havana,18,2,26479,0,0,73,26572
mirbase,0,0,1187,0,0,0,1187
All,29,15,53100,856,13,1401,55414


In [11]:
ensg.gene_df.coord_system.value_counts().head(10)

chromosome    55357
scaffold         57
Name: coord_system, dtype: int64

In [12]:
ensg.gene_df.gene_biotype.value_counts().head(10)

protein_coding            21884
processed_pseudogene       9999
lncRNA                     9949
TEC                        3237
unprocessed_pseudogene     2718
miRNA                      2206
snoRNA                     1507
snRNA                      1381
misc_RNA                    562
rRNA                        354
Name: gene_biotype, dtype: int64

In [13]:
ensg.gene_df.seq_region_exc_type.value_counts(dropna=False)

None    55414
Name: seq_region_exc_type, dtype: int64

In [14]:
ensg.gene_df.mhc.value_counts()

Series([], Name: mhc, dtype: int64)

In [15]:
len(ensg.gene_df)

55414

## alternative gene alleles

Related:

- [OTP: Origin of genes_with_non_reference_ensembl_ids.tsv](https://github.com/opentargets/platform/issues/702)
- [biostars: map between different assemblies of one ensembl release](https://www.biostars.org/p/143956/)
- using `attrib_type.code = "non_ref"` for `primary_assembly` doesn't appear to return any results

In [16]:
ensg.alt_allele_df.head()

,ensembl_gene_id,alt_allele_group_id,alt_allele_is_representative,primary_assembly,seq_region,alt_allele_attrib,ensembl_created_date,ensembl_representative_gene_id,is_representative_gene,representative_gene_method


In [17]:
# looks like non_ref isn't set for human genes
query = '''
SELECT *
FROM gene_attrib
LEFT JOIN attrib_type
  ON gene_attrib.attrib_type_id = attrib_type.attrib_type_id
WHERE attrib_type.code = "non_ref"
LIMIT 5
'''
pd.read_sql(sql=query, con=ensg.connection_url)

,gene_id,attrib_type_id,value,attrib_type_id,code,name,description


In [18]:
ensg.alt_allele_df.alt_allele_attrib.value_counts()

Series([], Name: alt_allele_attrib, dtype: int64)

In [19]:
ensg.alt_allele_df.query("is_representative_gene").representative_gene_method.value_counts()

Series([], Name: representative_gene_method, dtype: int64)

In [20]:
ensg.gene_df.query("ensembl_gene_id != ensembl_representative_gene_id").head(2)

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id


# replaced ID converter

A single `old_stable_id` can map to multiple `new_stable_id`. For example, `ENSG00000152006`

https://uswest.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=AzhM62SpkvdiLC4H-6808613

Requested ID | Matched ID(s) | Releases
-- | -- | --
ENSG00000152006 | ENSG00000196273 | 26: ENSG00000196273.1
ENSG00000152006 | ENSG00000197016 | 26: ENSG00000197016.1
ENSG00000152006 | ENSG00000196239 | 26: ENSG00000196239.1

In [21]:
ensg.old_to_new_df.head(2)

,old_ensembl_gene_id,new_ensembl_gene_id
0,ENSMUSG00000037155,ENSMUSG00000044465
1,ENSMUSG00000000700,ENSMUSG00000066358


In [22]:
# some ensembl genes replaced by many new ensembl genes
ensg.old_to_new_df.old_ensembl_gene_id.value_counts().head(2)

ENSMUSG00000070244    131
ENSMUSG00000070259    131
Name: old_ensembl_gene_id, dtype: int64

In [23]:
# example
ensg._update_ensembl_gene("ENSG00000152006")

{'ENSG00000152006'}

In [24]:
ensg.old_to_newest_df.head(2)

,old_ensembl_gene_id,newest_ensembl_gene_id,is_current
0,ENSMUSG00000000700,ENSMUSG00000066358,False
1,ENSMUSG00000000700,ENSMUSG00000068941,False


In [25]:
len(ensg.old_to_newest_df)

71291

In [26]:
ensg.old_to_newest_df.is_current.value_counts()

True     58574
False    12717
Name: is_current, dtype: int64

## omni-updater

The omni-updater dataset is designed to convert ensembl gene IDs from input data to the current, representative ensembl_gene_ids for this ensembl release. It assumes:

- users want to update outdated genes with their replacements
- users want a dataset of representative genes only, and want to convert alternative alleles to representative genes

An inner join of a dataset with `update_df` on `input_ensembl_gene_id` will do the following:

- produce output ensembl_gene_ids that are current and representatives
- update outdated genes with their current identifiers. Outdated genes with no current replacement will be removed by the inner join.
- update alternative gene alleles with their representatives
- genes that are already represenative and current will map to themselves

In [27]:
ensg.update_df.head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
0,ENSMUSG00000000001,ENSMUSG00000000001,True,True,1,1
1,ENSMUSG00000000003,ENSMUSG00000000003,True,True,1,1


In [28]:
ensg.update_df.sort_values("input_maps_to_n_genes", ascending=False).head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
34791,ENSMUSG00000075866,ENSMUSG00000119446,False,True,126,254
47269,ENSMUSG00000075808,ENSMUSG00000119506,False,True,126,160


In [29]:
ensg.update_df.sort_values("n_inputs_map_to_gene", ascending=False).head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
38857,ENSMUSG00000075878,ENSMUSG00000119694,False,True,122,255
41406,ENSMUSG00000075915,ENSMUSG00000119778,False,True,120,255


In [30]:
(ensg.update_df.input_maps_to_n_genes == 1).mean()

0.4937800470224936

In [31]:
ensg.update_df.query("ensembl_gene_id == 'ENSG00000256263'")

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene


In [32]:
print(
    f"The omni-updater contains {len(ensg.update_df):,} rows for mapping "
    f"{ensg.update_df.input_ensembl_gene_id.nunique():,} input genes to "
    f"{ensg.update_df.ensembl_gene_id.nunique():,} current, representative genes."
)

The omni-updater contains 113,988 rows for mapping 59,221 input genes to 55,414 current, representative genes.


In [33]:
# https://useast.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=P45VLMbogubpI0QA-6815464
ensg.update_df.query("input_ensembl_gene_id == 'ENSG00000201456'").head(3)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene


## cross-refrences (xrefs)

In [34]:
ensg.xref_df.head()

,ensembl_representative_gene_id,ensembl_gene_id,gene_symbol,xref_source,xref_accession,xref_label,xref_description,xref_info_type,xref_linkage_annotation,xref_curie
0,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,ArrayExpress,ENSMUSG00000000001,ENSMUSG00000000001,None,DIRECT,None,arrayexpress:ENSMUSG00000000001
1,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,EntrezGene,14679,Gnai3,guanine nucleotide binding protein (G protein)...,DEPENDENT,None,ncbigene:14679
2,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,MGI,MGI:95773,Gnai3,guanine nucleotide binding protein (G protein)...,DIRECT,None,mgi:95773
3,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,Reactome_gene,R-MMU-109582,R-MMU-109582,Hemostasis,DIRECT,None,reactome:R-MMU-109582
4,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,Reactome_gene,R-MMU-111885,R-MMU-111885,Opioid Signalling,DIRECT,None,reactome:R-MMU-111885


In [35]:
# datasets where there are ensembl_gene_id-xref_source-xref_accession pairs might not be distinct 
xref_dup_df = ensg.xref_df[ensg.xref_df.duplicated(subset=["ensembl_gene_id", "xref_source", "xref_accession"], keep=False)]
xref_dup_df.xref_source.value_counts()

Series([], Name: xref_source, dtype: int64)

In [36]:
# xref sources versus info_types
df = pd.crosstab(ensg.xref_df.xref_source, ensg.xref_df.xref_info_type, margins=True)
df["bioregistry_prefix"] = df.index.to_series().replace(ensg._xref_prefix_updater).map(normalize_prefix)
df

xref_info_type,DEPENDENT,DIRECT,MISC,PROJECTION,All,bioregistry_prefix
xref_source,,,,,,
ArrayExpress,0,55414,0,0,55414,arrayexpress
EntrezGene,27162,0,0,0,27162,ncbigene
HGNC,0,0,0,15,15,hgnc
MGI,0,53102,0,0,53102,mgi
RFAM,0,2468,0,0,2468,rfam
Reactome_gene,0,89896,0,0,89896,reactome
Uniprot_gn,52013,0,0,0,52013,uniprot
WikiGene,27162,0,0,0,27162,wikigenes
miRBase,0,0,1360,0,1360,mirbase


## Gene Ontology xrefs

In [37]:
ensg.xref_go_df.head(3)

,ensembl_gene_id,go_id,go_label,go_evidence_codes,xref_info_types,ensembl_transcript_ids,ensembl_representative_gene_id
0,ENSMUSG00000000001,GO:0000139,Golgi membrane,"IEA,ISO","PROJECTION,DIRECT",ENSMUST00000000001,ENSMUSG00000000001
1,ENSMUSG00000000001,GO:0000166,nucleotide binding,IEA,DIRECT,ENSMUST00000000001,ENSMUSG00000000001
2,ENSMUSG00000000001,GO:0001664,G protein-coupled receptor binding,IBA,DIRECT,ENSMUST00000000001,ENSMUSG00000000001


In [38]:
# GO terms for CCR5
# compare to http://useast.ensembl.org/Homo_sapiens/Gene/Ontologies/molecular_function?g=ENSG00000160791
sorted(ensg.xref_go_df.query("ensembl_gene_id == 'ENSG00000160791'").go_label)

[]

## lrg xrefs

In [39]:
ensg.xref_lrg_df.head(2)

,ensembl_gene_id,lrg_gene_id


In [40]:
len(ensg.xref_lrg_df)

0

### ncbigene xrefs

In [41]:
ensg.xref_ncbigene_df.head()

,ensembl_representative_gene_id,ncbigene_id,gene_symbol,ncbigene_symbol
1,ENSMUSG00000000001,14679,Gnai3,Gnai3
26,ENSMUSG00000000003,54192,Pbsn,Pbsn
33,ENSMUSG00000000028,12544,Cdc45,Cdc45
53,ENSMUSG00000000037,107815,Scml2,Scml2
64,ENSMUSG00000000049,11818,Apoh,Apoh


In [42]:
# ensembl gene mapped to by multiple ncbigenes
ensg.xref_ncbigene_df.ensembl_representative_gene_id.value_counts().head(3)

ENSMUSG00000000001    1
ENSMUSG00000067614    1
ENSMUSG00000067441    1
Name: ensembl_representative_gene_id, dtype: int64

In [43]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ensembl_representative_gene_id.duplicated().sum()

(26873, 0)

In [44]:
# ncbigene mapped to by multiple ensembl genes, likely due to alt gene alleles
ensg.xref_ncbigene_df.ncbigene_id.value_counts().head(3)

677884       3
100042024    2
16866        2
Name: ncbigene_id, dtype: int64

In [45]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ncbigene_id.duplicated().sum()

(26873, 23)

In [46]:
# ensg.xref_ncbigene_df.query("ensembl_representative_gene_id == 'ENSG00000231500'")
# ensg.xref_ncbigene_df.query("ncbigene_id == '51206'")

In [47]:
repr_ensembl_gene_ids = set(ensg.gene_df.ensembl_representative_gene_id)
len(repr_ensembl_gene_ids)

55414

In [48]:
# many of these genes should probably be alternative alleles rather than representative
ensg.gene_df.query("not primary_assembly and ensembl_gene_id==ensembl_representative_gene_id")

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
